In [1]:
from qmlify.openmm_torch.utils import prepare_ml_system

In [2]:
from qmlify.openmm_torch.utils import prepare_ml_system
from qmlify.openmm_torch.test_openmm_torch import get_HostGuestExplicit
import os
testsystem_class = get_HostGuestExplicit()

In [3]:
torch_scale_name = 'torch_scale'
torch_scale_default_value = 0.

In [4]:
ml_system, hybrid_factory = prepare_ml_system(                          
                          positions = testsystem_class.positions,
                          topology = testsystem_class.topology,
                          system = testsystem_class.system,
                          residue_indices = [1],
                          model_name='ani2x',
                          save_filename = 'repex.pt',
                          torch_scale_name=torch_scale_name,
                          torch_scale_default_value=torch_scale_default_value,
                          HybridSystemFactory_kwargs = {},
                          minimizer_kwargs = {'maxIterations': 1000}
                          )

INFO:utils:preparing ML system and initializing assertions...
INFO:utils:executing torch alchemification wrapper to make ml_system and hybrid_factory
INFO:force-hybridization:initializing system modifier...
INFO:force-hybridization:found 1 from [1]
INFO:force-hybridization:identified 30 to treat with ML.
INFO:force-hybridization:modifying harmonic bonds...
INFO:force-hybridization:modifying harmonic angles...
INFO:force-hybridization:modifying periodic torsions...
INFO:force-hybridization:modifying nonbondeds...
DEBUG:force-hybridization:iterating over existing exceptions and adding to custom force...
DEBUG:force-hybridization:iterating over alchemical particle combinations
DEBUG:force-hybridization:enabling rest-like scaling to alchemical particle nonbondeds
INFO:torchforce_generator:registering `torch` device...
INFO:torchforce_generator:found torch device 'cuda': False
DEBUG:torchforce_generator:elements: ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'C', 'O', 'H', 'H', 'H', 'H

In [5]:
from qmlify.openmm_torch.utils import configure_platform
from openmmtools import utils
from openmmtools.integrators import LangevinIntegrator
import simtk

In [6]:
from openmmtools import cache
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState


In [7]:
sampler_state = SamplerState(testsystem_class.positions, box_vectors=testsystem_class.system.getDefaultPeriodicBoxVectors())

In [8]:
from qmlify.openmm_torch.utils import *

In [9]:
thermostate = ThermodynamicState(ml_system, temperature = DEFAULT_TEMPERATURE)

In [10]:
DEFAULT_LAMBDA0s

{'lambda_MM_angles': 1.0,
 'lambda_MM_bonds': 1.0,
 'lambda_MM_torsions': 1.0,
 'lambda_electrostatic_scale': 0.0,
 'lambda_epsilon_scale': 0.0,
 'lambda_nonbonded_MM_electrostatics': 0.0,
 'lambda_nonbonded_MM_sterics': 0.0,
 'lambda_scale': 1.0}

In [11]:
class Protocol(object):

    def __init__(self, 
                 alchemical_lambda, 
                 T_low, 
                 T_target):
        from openmmtools.constants import kB
        import numpy as np
        beta0 = 1. / (kB * T_low)
        beta = 1. / (kB * T_target)
        
        functions = {'lambda_MM_bonds': lambda x, beta0, beta: 1 - x,
                         
                     'lambda_MM_angles': lambda x, beta0, beta: 1 - x,
                         
                     'lambda_MM_torsions': lambda x, beta0, beta: 1 - x,
                         
                     'lambda_electrostatic_scale': lambda x, beta0, beta: np.sqrt(beta / beta0) - 1.,
                         
                     'lambda_epsilon_scale': lambda x, beta0, beta: (beta / beta0) - 1.,
                         
                     'lambda_nonbonded_MM_electrostatics': lambda x, beta0, beta: x,
                         
                     'lambda_nonbonded_MM_sterics': lambda x, beta0, beta: x,
                         
                     'lambda_scale': lambda x, beta0, beta: beta / beta0,
                         
                     'torch_scale': lambda x, beta0, beta: x,
                         
                     'auxiliary_torch_scale': lambda x, beta0, beta : beta / beta0
                    }
        outters = {}
        for name, _function in functions.items():
            outters[name] = _function(alchemical_lambda, beta0, beta)
        
        self.lambdas = outters
        
        
        
        

In [12]:
from openmmtools.alchemy import AlchemicalState
class TorchAlchemicalState(AlchemicalState):
    """
    """
    
    #add the lambda_validator
    class _LambdaParameter(AlchemicalState._LambdaParameter):
        @staticmethod
        def lambda_validator(self, instance, parameter_value):
            if parameter_value is None:
                return parameter_value
            return float(parameter_value)
    
    # alchemical
    lambda_MM_bonds = _LambdaParameter('lambda_MM_bonds')
    lambda_MM_angles = _LambdaParameter('lambda_MM_angles')
    lambda_MM_torsions = _LambdaParameter('lambda_MM_torsions')
    lambda_electrostatic_scale = _LambdaParameter('lambda_electrostatic_scale')
    lambda_epsilon_scale = _LambdaParameter('lambda_epsilon_scale')
    lambda_nonbonded_MM_electrostatics = _LambdaParameter('lambda_nonbonded_MM_electrostatics')
    lambda_nonbonded_MM_sterics = _LambdaParameter('lambda_nonbonded_MM_sterics')
    lambda_scale = _LambdaParameter('lambda_scale')
    
    #ml
    torch_scale = _LambdaParameter('torch_scale')
    auxiliary_torch_scale = _LambdaParameter('auxiliary_torch_scale')
    
    def set_alchemical_parameters(self, lambda_protocol):
        for parameter_name in lambda_protocol:
            lambda_value = lambda_protocol[parameter_name]
            setattr(self, parameter_name, lambda_value)

    
    

In [13]:
import math
import numpy as np
num_replicas = 7
num_temps = num_replicas // 2 + 1  # Number of temperature replicas.
T_min = 300.0 * unit.kelvin  # Minimum temperature.
T_max = 800.0 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(num_temps-1)) - 1.0) / (math.e - 1.0)
    for i in range(num_temps)]
all_temperatures = temperatures + temperatures[::-1][1:]
alch_lambdas = np.linspace(0,1,num_replicas)

#all_temps = temperatures + temperatures[1:][::-1]

In [14]:
all_temperatures

[Quantity(value=300.0, unit=kelvin),
 Quantity(value=415.11860817409524, unit=kelvin),
 Quantity(value=575.7795681004795, unit=kelvin),
 Quantity(value=800.0, unit=kelvin),
 Quantity(value=575.7795681004795, unit=kelvin),
 Quantity(value=415.11860817409524, unit=kelvin),
 Quantity(value=300.0, unit=kelvin)]

In [15]:
lam_zero_alch_state = TorchAlchemicalState.from_system(ml_system)

In [16]:
compound_thermostate = CompoundThermodynamicState(thermostate, composable_states=[lam_zero_alch_state
                                                                                 ])

In [17]:
import copy

In [18]:
thermostate_list = []

In [19]:
for alch_lam, temp in zip(alch_lambdas, all_temperatures):
    comp_thermostate_copy = copy.deepcopy(compound_thermostate)
    protocol = Protocol(alch_lam, all_temperatures[0], temp).lambdas
    comp_thermostate_copy.set_alchemical_parameters(protocol)
    thermostate_list.append(comp_thermostate_copy)
                                                    

In [30]:
sampler_states = [copy.deepcopy(sampler_state) for i in range(num_replicas)]

In [31]:
import openmmtools

In [32]:
move = openmmtools.mcmc.LangevinSplittingDynamicsMove(n_steps=1, splitting='V0 V1 R O R V1 V0')

In [34]:
import tempfile

In [37]:
simulation = openmmtools.multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=2)

In [38]:
storage_path = tempfile.NamedTemporaryFile(delete=False).name + '.nc'
reporter = openmmtools.multistate.MultiStateReporter(storage_path, checkpoint_interval=1)

DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /tmp/tmp3q8mrinc_checkpoint.nc


In [39]:
simulation.create(thermodynamic_states=thermostate_list,
                   sampler_states=sampler_states,
                   storage=reporter)

DEBUG:mpiplus.mpiplus:Cannot find MPI environment. MPI disabled.
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x7f91553dd2e0>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x7f915a29c3a0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x7f915ffb89d0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x7f915ffb8ca0>
DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  63533B | 62.044KB | 0.061MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.088s
DEBUG:openmmtools.multistate.multistatesampler:Storing general ReplicaExchange options...
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x7f915ffb8dc0>
DEBUG:mpiplus.mpiplus:

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

In [40]:
simulation.run()

DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.


AlchemicalStateError: Could not find global parameter torch_scale in the system.